In [ ]:
import numpy as np
import csv
import os
from tqdm import tqdm

NPZ_DIRPATH = "drive/MyDrive/AngioAssessment/npz"
CSV_DIRPATH = "drive/MyDrive/AngioAssessment/csv"

FNAME = 'arr_0'
NUM_FILES = 5

data = []
counts = {"1": 0, "0": 0} # "1" == keyframed

def prep_training_data(npz_fpath, csv_fpath):
  csv_data = csv.reader(open(csv_fpath, 'r'))
  npz_data = np.load(npz_fpath)[FNAME]

  for index, anno in tqdm(csv_data):
    try:
      counts[anno] += 1
      data.append([npz_data[int(index)], np.eye(2)[int(anno)]])
    except Exception as e:
      ## not too sure why there is an extra annotation row
      print(e)

def compile_data():
  for i in range(1, 1+NUM_FILES):
    npz_fpath = os.path.join(NPZ_DIRPATH, f"npz-{i}.npz")
    csv_fpath = os.path.join(CSV_DIRPATH, f"csv-{i}.csv")
    prep_training_data(npz_fpath, csv_fpath)

compile_data()
np.random.shuffle(data) # shuffling the images